In [42]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [43]:
actions = np.array(['hello','i love you','my','Name','thanks','What','Your'])
DATA_PATH = os.path.join('MP_Data')
sequence_length = 30

In [44]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [45]:
label_map = {label:num for num, label in enumerate(actions)}

In [46]:
sequences, labels = [], []

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [47]:
X = np.array(sequences)

In [48]:
np.array(sequences).shape

(210, 30, 1662)

In [49]:
y = to_categorical(labels).astype(int)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard

In [52]:
# log_dir = os.path.join('Logs')
# tb_callback = TensorBoard(log_dir=log_dir)
# Define log directory
log_dir = os.path.join('Logs')

# Define TensorBoard callback
tb_callback = TensorBoard(log_dir=log_dir)

In [53]:


# Define the CNN model
model = Sequential()

# Add Convolutional layers
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(30, 1662)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to connect convolutional layers to dense layers
model.add(Flatten())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer
model.add(Dense(actions.shape[0], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks (if needed)
tensorboard = TensorBoard(log_dir='logs')

# Train the model
# model.fit(X_train, y_train, epochs=..., batch_size=..., callbacks=[tensorboard])


In [54]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
7/7 [==============================] - 2s 30ms/step - loss: 1.9663 - categorical_accuracy: 0.1407
Epoch 2/100
7/7 [==============================] - 0s 26ms/step - loss: 1.8614 - categorical_accuracy: 0.2513
Epoch 3/100
7/7 [==============================] - 0s 29ms/step - loss: 1.7448 - categorical_accuracy: 0.3216
Epoch 4/100
7/7 [==============================] - 0s 33ms/step - loss: 1.5770 - categorical_accuracy: 0.3970
Epoch 5/100
7/7 [==============================] - 0s 31ms/step - loss: 1.5148 - categorical_accuracy: 0.4271
Epoch 6/100
7/7 [==============================] - 0s 33ms/step - loss: 1.4805 - categorical_accuracy: 0.3769
Epoch 7/100
7/7 [==============================] - 0s 32ms/step - loss: 1.4075 - categorical_accuracy: 0.4322
Epoch 8/100
7/7 [==============================] - 0s 32ms/step - loss: 1.2000 - categorical_accuracy: 0.6030
Epoch 9/100
7/7 [==============================] - 0s 32ms/step - loss: 1.0847 - categorical_accuracy: 0.6131
Epoch 10/1

In [56]:
model.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x00000218F2F1C1D0>>

In [57]:
res = model.predict(X_test)

1/1 [==============================] - 0s 325ms/step


In [58]:
model.predict(X_test)

1/1 [==============================] - 0s 28ms/step


array([[2.99286773e-09, 2.85544388e-09, 2.83550471e-03, 9.96767879e-01,
        2.05916137e-08, 3.96555493e-04, 5.92163865e-08],
       [9.31604445e-01, 3.41869932e-09, 1.53949955e-07, 1.52149387e-11,
        6.83003590e-02, 5.58127886e-06, 8.94562763e-05],
       [5.50288532e-04, 3.29539986e-08, 5.37611631e-05, 2.32333791e-10,
        9.99382257e-01, 1.06296102e-05, 2.95315817e-06],
       [1.71615809e-01, 1.08168294e-07, 5.48722119e-06, 2.11063125e-10,
        8.28336775e-01, 1.85892659e-05, 2.32092771e-05],
       [9.99969482e-01, 4.76548627e-11, 4.98013714e-11, 1.18746777e-12,
        2.32051389e-05, 1.02624771e-08, 7.27202132e-06],
       [2.77740401e-05, 3.78429961e-11, 1.16380033e-06, 2.61856359e-04,
        1.01320587e-07, 9.99701083e-01, 7.98983547e-06],
       [1.08270690e-08, 1.10461633e-10, 6.14786728e-08, 4.10196699e-05,
        1.05833932e-10, 9.99958634e-01, 2.42395089e-07],
       [1.54505699e-06, 1.25786802e-02, 4.65781852e-07, 1.87096632e-07,
        1.40845886e-08, 2

In [59]:
actions[np.argmax(res[3])]

'thanks'

In [60]:
np.sum(res[2])

0.99999994

In [61]:
actions[np.argmax(y_test[2])]

'thanks'

In [62]:
actions[y_test[2]]

array(['hello', 'hello', 'hello', 'hello', 'i love you', 'hello', 'hello'],
      dtype='<U10')

In [63]:
actions[np.argmax(y_test[2])]

'thanks'

In [64]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [65]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 27ms/step


In [66]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)

0.8181818181818182

In [68]:
model.save('action6.h5')

In [83]:
del model

NameError: name 'model' is not defined